# M3 — Cuantización y validación (calidad vs rendimiento) con Olive + ONNX Runtime

## Objetivo
1. Crear un modelo ONNX pequeño **con pesos** (para que la cuantización tenga efecto).
2. Medir **tamaño** y una **latencia aproximada** (micro‑benchmark) en ORT.
3. Ejecutar un workflow de **Olive** que aplique cuantización ONNX (INT8) y genere un modelo nuevo.
4. Validar que la salida del modelo cuantizado sigue siendo “equivalente” (dentro de tolerancia).

## Prerrequisitos
- Ejecutar el notebook con el **kernel** de tu `.venv`.
- Paquetes: `olive-ai`, `onnxruntime`, `onnx`, `numpy`.

## Referencias oficiales (para el módulo)
- CLI: `olive run --config ...` y `python -m olive` si `olive` no está en PATH. (Quick Tour)  
- Config `input_model`: `{ "type": "...ModelHandler", "config": {...} }` y soporta `ONNXModelHandler`. (Options)  
- Cuantización ONNX: `OnnxQuantization` y también `OnnxDynamicQuantization` / `OnnxStaticQuantization`. (Quantization)  
- Pass `OnnxDynamicQuantization` incluye `quant_mode` (default `dynamic`) y `weight_type` (default `QInt8`). (Passes)


In [1]:
# Celda 1 — Chequeo de entorno (siempre)
import sys, subprocess
import onnxruntime as ort

print("Python executable:", sys.executable)
print("Python version:", sys.version)
print("ONNX Runtime:", ort.get_version_string())
print("Available providers:", ort.get_available_providers())

print("\n--- pip show olive-ai ---")
subprocess.run([sys.executable, "-m", "pip", "show", "olive-ai"], check=False)


Python executable: g:\source\VisualCode\repos\olive-python-vscode-labs\.venv\Scripts\python.exe
Python version: 3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]
ONNX Runtime: 1.23.2
Available providers: ['AzureExecutionProvider', 'CPUExecutionProvider']

--- pip show olive-ai ---


CompletedProcess(args=['g:\\source\\VisualCode\\repos\\olive-python-vscode-labs\\.venv\\Scripts\\python.exe', '-m', 'pip', 'show', 'olive-ai'], returncode=0)

## 1) Crear un modelo ONNX con pesos (Linear: Y = X·W + b)

Creamos un ONNX con `MatMul` + `Add` y guardamos en `models/linear_fp32.onnx`.

- Input: `X` shape `[batch, in_features]` (batch dinámico)
- Pesos: `W` shape `[in_features, out_features]`
- Bias: `b` shape `[out_features]`


In [2]:
from pathlib import Path
import numpy as np
import onnx
from onnx import TensorProto, helper, numpy_helper

Path("../models").mkdir(exist_ok=True)
Path("../outputs").mkdir(exist_ok=True)

in_features = 4
out_features = 3

# IO
X = helper.make_tensor_value_info("X", TensorProto.FLOAT, ["batch", in_features])
Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, ["batch", out_features])

# Pesos y bias (float32)
rng = np.random.default_rng(0)
W_val = rng.standard_normal((in_features, out_features), dtype=np.float32)
b_val = rng.standard_normal((out_features,), dtype=np.float32)

W = numpy_helper.from_array(W_val, name="W")
b = numpy_helper.from_array(b_val, name="b")

# Nodos: MatMul(X, W) -> Z ; Add(Z, b) -> Y
matmul = helper.make_node("MatMul", inputs=["X", "W"], outputs=["Z"])
add = helper.make_node("Add", inputs=["Z", "b"], outputs=["Y"])

graph = helper.make_graph(
    nodes=[matmul, add],
    name="linear",
    inputs=[X],
    outputs=[Y],
    initializer=[W, b],
)

# Para compatibilidad amplia: opset 11 e IR 11
opset = [helper.make_operatorsetid("", 11)]
model = helper.make_model(graph, producer_name="m3-lab", opset_imports=opset)
model.ir_version = 11

onnx.checker.check_model(model)

fp32_model_path = Path("../models") / "linear_fp32.onnx"
onnx.save_model(model, str(fp32_model_path))

fp32_model_path


WindowsPath('../models/linear_fp32.onnx')

## 2) Baseline con ONNX Runtime: validar y medir (aprox)

Medimos:
- tamaño del archivo (`bytes`)
- tiempo medio por inferencia (simple micro‑benchmark)

> Nota: esto es una medida aproximada para comparar “antes vs después”.


In [3]:
import time
from pathlib import Path
import numpy as np
import onnxruntime as ort

def run_and_time(model_path: Path, x: np.ndarray, iters: int = 200, warmup: int = 20):
    sess = ort.InferenceSession(str(model_path), providers=["CPUExecutionProvider"])
    inp = sess.get_inputs()[0].name
    out = sess.get_outputs()[0].name

    # warmup
    for _ in range(warmup):
        sess.run([out], {inp: x})

    t0 = time.perf_counter()
    for _ in range(iters):
        y = sess.run([out], {inp: x})[0]
    t1 = time.perf_counter()

    avg_ms = (t1 - t0) * 1000 / iters
    return y, avg_ms

x = np.array([[1.0, 2.0, 3.0, 4.0]], dtype=np.float32)

y_fp32, fp32_ms = run_and_time(fp32_model_path, x)
fp32_size = fp32_model_path.stat().st_size

print("FP32 model:", fp32_model_path)
print("Size (bytes):", fp32_size)
print("Avg latency (ms):", fp32_ms)
print("y_fp32:", y_fp32)


FP32 model: ..\models\linear_fp32.onnx
Size (bytes): 198
Avg latency (ms): 0.005691999999726249
y_fp32: [[ 5.2784495  1.9592975 -5.466373 ]]


## 3) Workflow Olive: cuantización ONNX (dinámica, INT8)

En este laboratorio usamos `OnnxDynamicQuantization` (modo `dynamic`) para evitar calibración/datasets.


In [4]:
import json
from pathlib import Path

# Resolver raíz del repo (si el notebook está en notebooks/, sube un nivel)
root = Path.cwd()
if not (root / "models" / "linear_fp32.onnx").exists() and (root.parent / "models" / "linear_fp32.onnx").exists():
    root = root.parent

model_path = root / "models" / "linear_fp32.onnx"
output_dir = root / "outputs" / "m3_linear_int8"
cache_dir = root / "outputs" / "m3_cache"
config_path = root / "outputs" / "m3_run_config.json"

(root / "outputs").mkdir(exist_ok=True)

config = {
    "workflow_id": "m3_linear_quant_dynamic",
    "input_model": {
        "type": "ONNXModel",
        "config": {"model_path": str(model_path)},
    },
    "systems": {
        "local_system": {
            "type": "LocalSystem",
            "config": {
                "accelerators": [
                    {"device": "cpu", "execution_providers": ["CPUExecutionProvider"]}
                ]
            },
        }
    },
    "passes": {
        "quant_int8": {
            "type": "OnnxDynamicQuantization",
            "config": {
            "quant_mode": "dynamic",
            "weight_type": "QInt8"
            }
        }
    },
    "engine": {
        "host": "local_system",
        "target": "local_system",
        "cache_dir": str(cache_dir),
        "output_dir": str(output_dir),
        "log_severity_level": 0,
        "evaluate_input_model": False,
    },
}

config_path.write_text(json.dumps(config, indent=2), encoding="utf-8")
print("Wrote:", config_path)
print("Output dir:", output_dir)


Wrote: g:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_run_config.json
Output dir: g:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_linear_int8


## 4) Ejecutar Olive

Usamos `sys.executable -m olive ...` para asegurar que se ejecuta con el Python del kernel (tu `.venv`).


In [5]:
import sys, subprocess

cmd = [sys.executable, "-m", "olive", "run", "--config", str(config_path)]
print("Running:", " ".join(cmd))

completed = subprocess.run(cmd, text=True, capture_output=True)
print("returncode:", completed.returncode)
print("---- stdout ----")
print(completed.stdout[-4000:])  # último tramo para no saturar
print("---- stderr ----")
print(completed.stderr[-4000:])

if completed.returncode != 0:
    raise RuntimeError("Olive falló. Revisa stdout/stderr arriba.")


Running: g:\source\VisualCode\repos\olive-python-vscode-labs\.venv\Scripts\python.exe -m olive run --config g:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_run_config.json
returncode: 0
---- stdout ----
rkflow m3_linear_quant_dynamic
[2026-01-12 10:26:35,422] [INFO] [cache.py:138:__init__] Using cache directory: G:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_cache\m3_linear_quant_dynamic
[2026-01-12 10:26:35,422] [DEBUG] [cache.py:274:cache_olive_config] Cached olive config to G:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_cache\m3_linear_quant_dynamic\olive_config.json
[2026-01-12 10:26:35,426] [DEBUG] [accelerator_creator.py:106:_fill_accelerators] The accelerator device and execution providers are specified, skipping deduce.
[2026-01-12 10:26:35,427] [DEBUG] [accelerator_creator.py:143:_check_execution_providers] Supported execution providers for device cpu: [<ExecutionProvider.CPUExecutionProvider: 'CPUExecutionProvider'>, <ExecutionPr

## 5) Localizar el modelo cuantizado generado

In [6]:
from pathlib import Path

out = output_dir
onnx_files = sorted(out.rglob("*.onnx"))
print("ONNX files:", len(onnx_files))
for p in onnx_files:
    print("-", p.relative_to(out))

if not onnx_files:
    raise FileNotFoundError(f"No encuentro .onnx bajo {out}. Revisa logs de Olive.")

int8_model_path = onnx_files[0]
print("\nSelected:", int8_model_path)


ONNX files: 1
- model.onnx

Selected: g:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_linear_int8\model.onnx


## 6) Validación numérica + comparación de tamaño/latencia

In [7]:
import numpy as np

y_int8, int8_ms = run_and_time(int8_model_path, x)
int8_size = int8_model_path.stat().st_size

print("INT8 model:", int8_model_path)
print("Size (bytes):", int8_size)
print("Avg latency (ms):", int8_ms)
print("y_int8:", y_int8)

print("\nAllclose(fp32, int8):", np.allclose(y_fp32, y_int8, rtol=1e-02, atol=1e-03))
print("Max abs diff:", float(np.max(np.abs(y_fp32 - y_int8))))
print("Size reduction (%):", (1 - int8_size / fp32_size) * 100)


INT8 model: g:\source\VisualCode\repos\olive-python-vscode-labs\outputs\m3_linear_int8\model.onnx
Size (bytes): 1180
Avg latency (ms): 0.004530999999587948
y_int8: [[ 5.2475195  1.9376892 -5.4539256]]

Allclose(fp32, int8): False
Max abs diff: 0.030930042266845703
Size reduction (%): -495.95959595959596


## Verificación (checklist)
- `outputs/m3_linear_int8/` contiene al menos un `.onnx`.
- El modelo INT8 carga con ORT (sin errores).
- `np.allclose(...)` es `True` (o el error es pequeño).
- (Opcional) el tamaño del archivo baja y/o la latencia mejora.

## Siguiente paso (M4)
Medición más seria (latencia/throughput), empaquetado y preparación para hardware objetivo.
